### Задание 1

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import pingouin as pg
from scipy.stats import chi2_contingency, chi2 

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from operator import attrgetter
from matplotlib import colors as mcolors

In [2]:
#Открываем первый файл
df1 = pd.read_csv('~/shared/problem1-reg_data.csv', sep=';')
df1

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5
...,...,...
999995,1600874034,1110618
999996,1600874086,1110619
999997,1600874139,1110620
999998,1600874191,1110621


In [3]:
#Открываем второй файл
df2 = pd.read_csv('~/shared/problem1-auth_data.csv', sep=';')
df2

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2
...,...,...
9601008,1600874034,1110618
9601009,1600874086,1110619
9601010,1600874139,1110620
9601011,1600874191,1110621


In [4]:
#Соединим все таблицы в одну
df = df2.merge(df1, how='left', on='uid')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9601013 entries, 0 to 9601012
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
 2   reg_ts   int64
dtypes: int64(3)
memory usage: 293.0 MB


In [5]:
#Переводим время регистрации и время захода пользователей в игру в формат datetime64 
df['auth_ts'] = pd.to_datetime(df['auth_ts'], unit='s')
df['reg_ts'] = pd.to_datetime(df['reg_ts'], unit='s')


In [ ]:
#Извлекаем день из времени регистрации и времени захода пользователей в игру
df ['reg_date'] = df['reg_ts'].dt.to_period('D')
df ['auth_date'] = df['auth_ts'].dt.to_period('D')


In [ ]:
#Создаем отдельную колонку с количеством дней от времени регистрации
df['number_days'] = ((df.auth_date - df.reg_date)/ np.timedelta64(1, 'D'))


In [ ]:
#Агрегируем данные по когортам и рассчитанному количеству дней от времени регистрации
df_cohort = df.groupby(['reg_date', 'number_days']).agg({'uid':'nunique'}).reset_index()


In [ ]:
#Создаем сводную таблицу для когортного анализа 
cohort_pivot = df_cohort.pivot_table(index='reg_date', columns = 'number_days', values = 'uid')


In [ ]:
#Рассчитываем размеры когорт (первый столбец сводной таблицы)
cohort_size = cohort_pivot.iloc[:,0]


In [ ]:
#Вычисляем коэффициенты удержания, деля на размер когорты
retention_matrix = cohort_pivot.divide(cohort_size, axis =0)
retention_matrix

In [ ]:
with sns.axes_style("white"):
    fig, ax=plt.subplots(1,2, figsize=(16,12),sharey=True, gridspec_kw={'width_ratios':[1,11]})
    #Тепловая карта для коээфициентов удержания
    sns.heatmap(retention_matrix,
               mask=retention_matrix.isnull(),
               annot=True,
               fmt='.0%',
               cmap='RdYlGn',
               ax=ax[1])
    ax[1].set_title('Daily Retention', fontsize=16)
    ax[1].set(xlabel='Количество дней после регистрации ', ylabel='Дата регистрации')
    #Тепловая карта для размеров когорт
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0:'cohort_size'})
    white_cmap=mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df,
               annot=True,
               cbar= False,
               fmt='g',
               cmap=white_cmap,
               ax=ax[0])
    fig.tight_layout()
    plt.show()

In [ ]:
#Создаем функцию
def retention(df1, df2, Beginning, End):
    '''Функция проводит когортный анализ, визуализирует его с помощью тепловой карты и рассчитывает retention игроков (по дням от даты регистрации игрока) на любом срезе времени'''
    df = df2.merge(df1, how='left', on='uid')
    Beginning = pd.to_datetime(Beginning, unit='s').dt.to_period('D')
    End = pd.to_datetime(End, unit='s').dt.to_period('D')
    df ['reg_date'] = df['reg_ts'].dt.to_period('D')
    df ['auth_date'] = df['auth_ts'].dt.to_period('D')
    #Добавим условие, чтобы мы могли использовать функцию на части данных 
    df = df.query('reg_ts >= Beginning and auth_ts <= End')
    df['number_days'] = ((df.auth_date - df.reg_date)/ np.timedelta64(1, 'D'))
    df_cohort = df.groupby(['reg_date', 'number_days']).agg({'uid':'nunique'}).reset_index()\
    .pivot_table(index='reg_date', columns = 'number_days', values = 'uid')
    cohort_size = cohort_pivot.iloc[:,0]
    retention_matrix = cohort_pivot.divide(cohort_size, axis =0)
    with sns.axes_style("white"):
        fig, ax=plt.subplots(1,2, figsize=(16,12),sharey=True, gridspec_kw={'width_ratios':[1,11]})
    #Тепловая карта для коээфициентов удержания
        sns.heatmap(retention_matrix,
               mask=retention_matrix.isnull(),
               annot=True,
               fmt='.0%',
               cmap='RdYlGn',
               ax=ax[1])
        ax[1].set_title('Daily Retention', fontsize=16)
        ax[1].set(xlabel='Количество дней после регистрации ', ylabel='Дата регистрации')
    #Тепловая карта для размеров когорт
        cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0:'cohort_size'})
        white_cmap=mcolors.ListedColormap(['white'])
        sns.heatmap(cohort_size_df,
               annot=True,
               cbar= False,
               fmt='g',
               cmap=white_cmap,
               ax=ax[0])
        fig.tight_layout()
        plt.show()
    return sns.axes_style("white")

# Задание 2

In [ ]:
#Открываем файл 
df = pd.read_csv('Проект_1_Задание_2.csv', sep=';')
df

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.describe()

In [ ]:
#Высчитываем количество платящих пользователей для каждой группы
paying_users = df.query('revenue!=0').groupby(['testgroup']).agg(paying_users =('user_id','count'))
paying_users

Создаем отдельную таблицу с показателями отдельно для каждой группы. Рассчитываем:

* Средний чек (ARPU) 
* Количество пользователей (user_id) 
* Количество платящих пользователей (paying_users) 
* Доход (revenue)
* Средний чек по платящему клиенту (ARPPU)
* Конверсия в платящих пользователей (CR)

In [ ]:
value = df.groupby(['testgroup']).agg({'revenue': 'sum', 'user_id':'count'}).reset_index()
value ['ARPU'] = value['revenue']/value['user_id']
value ['ARPU'] = value ['ARPU'].round(2)
value = value.merge(paying_users, on='testgroup', how='right')
value ['ARPPU'] = value['revenue']/value['paying_users']
value ['ARPPU'] = value ['ARPPU'].round(2)
value ['CR'] = (value['paying_users']/value['user_id'])*100
value

По условию: в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

Значит, а - контрольная группа, b - тестовая группа.

Также видим, что показатели ARPPU и ARPU и доход больше в тестовой группе, но при этом конверсия и количество платящих пользователей больше в контрольной группе.

In [ ]:
#Создаем для каждой группы отдельную таблицу для удобства
df_a = df.query('testgroup=="a"')
df_b = df.query('testgroup=="b"')

##### Проверим гипотезу о взаимосвязи метрики ARPU между тестовой и контрольной группой. 

𝐻0 : ARPU в группах равны

𝐻1 : ARPU в группах не равны

Для этого проверим является ли распределение переменной ARPU нормальной.

In [ ]:
# Проверим нормальность распределения по Шапиро
pg.normality(data=df, dv="ARPU", group="testgroup")

In [ ]:
# по альтернативному тесту
pg.normality(data=df, dv="ARPU", group="testgroup", method="normaltest")

Так как распределение не является нормальным, но при этом выборка очень большая для проверки гипотезы можно воспользоваться t-критерием с поправкой Уэлча.

In [ ]:
st.ttest_ind(df_a.revenue, df_b.revenue, equal_var=False)

Так как p-value > 0.05, то ARPU в группах равны.

##### Проверим гипотезу о взаимосвязи конверсии в платящих пользователей (CR) между тестовой и контрольной группой. 
Проверка на нормальность не требуется, т.к. проверяются категориальные переменные.

Так как переменная категориальная воспользуемся методом Хи-квадрат.

𝐻0 : CR в группах равны

𝐻1 : CR в группах не равны

In [ ]:
#Создаем колонку, в которой показано совершил ли пользователь платеж 
df['Payment'] = df.revenue.apply(lambda x: x > 0)

In [ ]:
stat, p, dof, expected = chi2_contingency(pd.crosstab(df.Payment, df.testgroup))

In [ ]:
stat, p

In [ ]:
prob = 0.95
critical = chi2.ppf(prob, dof)
if abs(stat) >= critical:
    print('Отклоняем H0')
else:
    print('Не отклоняем H0')

##### Проверим гипотезу о взаимосвязи метрики ARPPU между тестовой и контрольной группой. 

𝐻0 : ARPPU в группах равны

𝐻1 : ARPPU в группах не равны

Не имеет смысла проверять нормальность переменной ARPPU, так как у выборки большой объем. Проверить гипотезу можно с помощью t-критерием с поправкой Уэлча.

In [ ]:
st.ttest_ind(df_a[df_a["revenue"] > 0]["revenue"], df_b[df_b["revenue"] > 0]["revenue"], equal_var=False)

In [ ]:
Так как p-value > 0.05, то ARPPU в группах равны.

# Выводы

По результатам анализа ARPPU и ARPU в группах равны, то есть не оказывается значительного влияния на доход от приложения. При этом конверсия (статистически значимый показатель) у контрольной группы больше чем у тестовой. Можно сделать вывод, что набор акционных предложений в контрольной группе лучше чем в тестовой. 

# Задание 3

В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

Для оценки результатов последнего прошедшего события можно рассмотреть  следующие метрики(так как в условиях не указано ничего о цене и платежах игроков во время события, финансовые метрики не берем в расчет):
1. DAU (Daily Active Users) — количество уникальных игроков за сутки;
2. Retention - какая доля игроков вернулась в продукт во время тематического события;
3. Churn rate — сколько игроков перестало играть за время тематического события;
4. CSAT (Customer Satisfaction Score)-  удовлетворенность игроков(можно провести опрос).

При усложнении механики событий набор метрик для оценки результата не изменится. Но также можно провести A/B тест и сравнить все написанные выше метрики.